In this notebook, we thin the MCMC chains to creat shorter model posterior samples to perform the kinematic computation on. We keep 10,000 samples from each model setup.

### import libraries

In [1]:
import numpy as np
import os
import pickle
import copy
import time
import h5py
from pprint import pprint
import joblib
import gc

import lenstronomy.Util.util as util
from lenstronomy.Workflow.fitting_sequence import FittingSequence
from lenstronomy.Plots.model_plot import ModelPlot
from lenstronomy.Plots import chain_plot
from lenstronomy.Sampling.parameters import Param
#from lenstronomy.Analysis.lens_analysis import LensAnalysis
from lenstronomy.Util import mask_util
from lenstronomy.Data.coord_transforms import Coordinates
from lenstronomy.Util.util import array2image

cwd = os.getcwd()
base_path, _ = os.path.split(cwd)
print('Base directory path:', base_path)

Base directory path: /Users/ajshajib/Research/time_delay_cosmography/J2038/WGD2038-4008/lenstronomy_modeling


## output analysis

In [2]:
def custom_loglikelihood_addition(kwargs_lens=None, kwargs_source=None, 
                                 kwargs_lens_light=None, kwargs_ps=None, 
                                 kwargs_special=None, kwargs_extinction=None):
    """
    Impose a Gaussian prior on the NFW scale radius R_s based on Gavazzi et al. (2007).
    """
    # imports inside function to avoid pickling 
    from colossus.halo import concentration
    from colossus.halo import mass_defs
    from colossus.cosmology import cosmology
    
    from lenstronomy.Cosmo.lens_cosmo import LensCosmo
    from lenstronomy.Analysis.lens_profile import LensProfileAnalysis
    from lenstronomy.LensModel.lens_model import LensModel


    lens_cosmo = LensCosmo(z_lens=0.230, z_source=0.777)
    lens_analysis = LensProfileAnalysis(
                            LensModel(lens_model_list=['NFW_ELLIPSE', 'SHEAR', 'TRIPLE_CHAMELEON'],
                            z_lens=0.230, z_source=0.777,
                            multi_plane=False, #True,
                            ))


    if kwargs_lens[0]['alpha_Rs'] < 0.:
        return -np.inf
    
    if not -0.014271818911080656-0.2 < kwargs_lens[0]['center_x'] < -0.014271818911080656+0.2:
        return -np.inf
    if not -0.020882886550870693-0.2 < kwargs_lens[0]['center_y'] < -0.020882886550870693+0.2:
        return -np.inf
    
    if not -0.5 < kwargs_lens[0]['e1'] < 0.5:
        return -np.inf
    if not -0.5 < kwargs_lens[0]['e2'] < 0.5:
        return -np.inf
    
    log_L = 0.
    
    log_L += -0.5 * (kwargs_lens[0]['Rs'] - 22.6)**2 / 3.1**2
    
    # integrate upto 3.2 arcsec, which is half-light radius (~half-mass radius)
#     mean_convergence = lens_analysis.mass_fraction_within_radius(kwargs_lens, 
#                                                              kwargs_lens[2]['center_x'], 
#                                                              kwargs_lens[2]['center_y'], 
#                                                              3.2,
#                                                              numPix=320)
    
#     stellar_mass = np.log10(mean_convergence[2] * np.pi * (3.2/206265 * lens_cosmo.dd)**2 
#                             * lens_cosmo.sigma_crit * 2) # multiplying by 2 to convert half-mass to full mass
    
    #log_L += - 0.5 * (stellar_mass - 11.40)**2 / (0.08**2 + 0.1**2)
    # adding 0.07 uncertainty in quadrature to account for 15% uncertainty in H_0, Om_0 ~ U(0.05, 0.5)
#     high_sm = 11.57 + 0.25 + 0.06 # +0.06 is to add H_0 uncertainty
#     low_sm = 11.57 - 0.06 # -0.06 is to add H_0 uncertainty
#     if stellar_mass > high_sm:
#         log_L += -0.5 * (high_sm - stellar_mass)**2 / (0.16**2)
#     elif stellar_mass < low_sm:
#         log_L += -0.5 * (low_sm - stellar_mass)**2 / (0.13**2)
#     else:
#         log_L += 0.
        
    _, _, c, r, halo_mass = lens_cosmo.nfw_angle2physical(kwargs_lens[0]['Rs'], kwargs_lens[0]['alpha_Rs'])
#     log_L += -0.5 * (np.log10(halo_mass) - 13.5)**2 / 0.3**2
    
    my_cosmo = {'flat': True, 'H0': 70., 'Om0': 0.3, 'Ob0': 0.05, 
                'sigma8': 0.823, 'ns': 0.96} # fiducial cosmo
    cosmo = cosmology.setCosmology('my_cosmo', my_cosmo)

    c200 = concentration.concentration(halo_mass*cosmo.h, '200c', # input halo mass needs to be in M_sun/h unit
                                       0.23, model='diemer19')
    
    log_L += -0.5 * (np.log10(c) - np.log10(c200))**2 / (0.11**2)
            
    return log_L

In [3]:
# ssh_command = 'ajshajib@hoffman2.idre.ucla.edu'
# dir_path = os.getcwd()
# dir_path_cluster = '/u/flashscratch/a/ajshajib/'


output_file_list = [
    '2038_run214_1_0_0_0_0',
    '2038_run214_1_0_1_0_0',
    '2038_run214_1_0_2_0_0',
    '2038_run214_1_1_0_0_0',
    '2038_run214_1_1_1_0_0',
    '2038_run214_1_1_2_0_0',
    
    '2038_run215_1_0_0_0_0',
    '2038_run215_1_0_1_0_0',
    '2038_run215_1_0_2_0_0',
    '2038_run215_1_1_0_0_0',
    '2038_run215_1_1_1_0_0',
    '2038_run215_1_1_2_0_0',
]

for job_name_out in output_file_list:
    gc.collect()
    print("Processing: ", job_name_out)

    #hoffman2_out = False
    input_temp = os.path.join(base_path, 'temp', job_name_out +'.txt')
    output_temp = os.path.join(base_path, 'temp', job_name_out +'_out.txt')

    f = open(output_temp, 'rb')
    [input_, output_] = joblib.load(f)
    f.close()
    
    fitting_kwargs_list, multi_band_list, kwargs_model, kwargs_constraints, kwargs_likelihood, kwargs_params, init_samples = input_

    kwargs_result, multi_band_list_out, fit_output, _ = output_

    print('Number of fitting sequence:', len(fit_output))

    print('Number of params:', len(fit_output[-1][2]))

    print("fit_output sequence before trimming...")
    
    for a in range(len(fit_output)):
        print(fit_output[a][0])
        
    for i in range(1):
        del fit_output[-2]

    #for i in range(4):
    #    del fit_output[-3]

    print("fit_output sequence after trimming...")
    for a in range(len(fit_output)):
        print(fit_output[a][0])

    samples_mcmc = fit_output[-1][1]

    n_params = samples_mcmc.shape[1]

    n_walkers = 8 * n_params
    n_step = int(samples_mcmc.shape[0] / n_walkers)

    print('n_step: {}, n_walkers: {}, n_params: {}'.format(n_step, n_walkers, n_params))

    chain = np.empty((n_walkers, n_step, n_params))

    for i in np.arange(n_params):
        samples = samples_mcmc[:, i].T
        chain[:,:,i] = samples.reshape((n_step, n_walkers)).T
    
    print(chain.shape)
    burnt_in = -300
    thin = 10
    
    random_indices = np.random.randint(low=0, high=n_params*np.abs(burnt_in)/thin, size=10000)
    
    samples_mcmc = chain[:, burnt_in::thin, :].reshape((-1, n_params))[random_indices]
    likelihoods_new = fit_output[-1][-1].reshape((n_step, n_walkers))[burnt_in::thin, :].flatten()[random_indices]
    
    print('Trimmed MCMC chain from:', fit_output[-1][1].shape, 'to: ', samples_mcmc.shape)
    print('Trimmed likelihood chain from:', fit_output[-1][-1].shape, 'to: ', likelihoods_new.shape)
    
    fit_output[-1][1] = samples_mcmc
    
    fit_output_new = [[]]
    for i in range(len(fit_output[-1])):
        if i == 1:
            fit_output_new[0].append(samples_mcmc)
        if i == len(fit_output[-1]) - 1:
            fit_output_new[0].append(likelihoods_new)
        else:
            fit_output_new[0].append(fit_output[-1][i])

    modified_file_path = os.path.join(base_path, 'temp', job_name_out +'_shortened_out.txt')
    
    for kw in fitting_kwargs_list:
        kw[1]['init_samples'] = None
        
    input_ = fitting_kwargs_list, multi_band_list, kwargs_model, kwargs_constraints, kwargs_likelihood, kwargs_params, None

    output_ = kwargs_result, multi_band_list_out, fit_output, None

    modified_output = [input_, output_]
    f = open(modified_file_path, 'wb')
    joblib.dump(modified_output, f)
    f.close()
    
    print('\n')

Processing:  2038_run214_1_0_0_0_0
Number of fitting sequence: 2
Number of params: 54
fit_output sequence before trimming...
EMCEE
EMCEE
fit_output sequence after trimming...
EMCEE
n_step: 1000, n_walkers: 432, n_params: 54
(432, 1000, 54)
Trimmed MCMC chain from: (432000, 54) to:  (10000, 54)
Trimmed likelihood chain from: (432000,) to:  (10000,)


Processing:  2038_run214_1_0_1_0_0
Number of fitting sequence: 2
Number of params: 54
fit_output sequence before trimming...
EMCEE
EMCEE
fit_output sequence after trimming...
EMCEE
n_step: 1000, n_walkers: 432, n_params: 54
(432, 1000, 54)
Trimmed MCMC chain from: (432000, 54) to:  (10000, 54)
Trimmed likelihood chain from: (432000,) to:  (10000,)


Processing:  2038_run214_1_0_2_0_0
Number of fitting sequence: 2
Number of params: 54
fit_output sequence before trimming...
EMCEE
EMCEE
fit_output sequence after trimming...
EMCEE
n_step: 1000, n_walkers: 432, n_params: 54
(432, 1000, 54)
Trimmed MCMC chain from: (432000, 54) to:  (10000, 54)
T